# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
list(Path('/content/drive').iterdir())

[PosixPath('/content/drive/.shortcut-targets-by-id'),
 PosixPath('/content/drive/.file-revisions-by-id'),
 PosixPath('/content/drive/Shareddrives'),
 PosixPath('/content/drive/MyDrive'),
 PosixPath('/content/drive/.Trash')]

# Imports 

In [ ]:
import numpy as np 
import pandas as pd

from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import itertools
import nltk 
import re                    
from nltk.corpus import stopwords  
from gensim import parsing       
import gensim
from matplotlib import pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.utils import Sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing import sequence
from keras.callbacks import ReduceLROnPlateau

import math
import os
import pandas as pd
import glob
import time
import cv2

#Combining the files containing the xssed and normal samples.

path = '/content/drive/DSCI' # use your path
#all_files = glob.glob(path + "/*.csv")
all_files =['/content/drive/MyDrive/DSCI/normal_examples.csv','/content/drive/MyDrive/DSCI/xssed.csv']
li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0, skipinitialspace=True)
    li.append(frame)

#np.random.shuffle(li)
df = pd.concat(li, axis=0, ignore_index=True)


### Getting the dataframe

In [ ]:
print(li)

[                                                    Code  Label
0                sid=&amp;ring=hentff98&amp;id=&amp;list      0
1                                            ring=yaoi28      0
2                    mode=navigation&amp;categoryid=1141      0
3                                                lang=en      0
4                                            ring=bettie      0
...                                                  ...    ...
31402  sgwid=4-102-70-1066702-detailspage=journal|des...      0
31403                                       subject=ls60      0
31404                                            jid=grh      0
31405                         journal=mcg&amp;page=index      0
31406                           path=228;273;280;428;287      0

[31407 rows x 2 columns],                                                     Code  Label
0      Search%3D%3C/script%3E%3Cimg/%2A%00/src%3D%22w...      1
1      symbol%3D%3Ch1%3E%3Cscript%3Ealert%28/hacked/%...      1
2      query

In [ ]:
df.head()

,Code,Label
0,sid=&amp;ring=hentff98&amp;id=&amp;list,0
1,ring=yaoi28,0
2,mode=navigation&amp;categoryid=1141,0
3,lang=en,0
4,ring=bettie,0


In [ ]:
del li
import gc 
gc.collect()

47

In [ ]:
#Shuffling the normal ad xss samples to get a mixed dataset. 
from sklearn.utils import shuffle
df= shuffle(df, random_state=0)

In [ ]:
df

,Code,Label
73039,clef=crochesdor&titre=%22%3E%3Cscript%3Ealert%...,1
4960,aid=/20011207/reviews/112070301/1023,0
96239,prod=%22%3E%27%3E%3C%73%63%72%69%70%74%3E%61%6...,1
60741,lg%3D%3C/script%3E%3Ciframe%20src%3Dhttp%3A//x...,1
71075,country%3D%22%3E%3Cscript%3Ealert%28document.c...,1
...,...,...
21243,publicationid=731,0
45891,offset%3D5%26sortby%3Dlivedate%20desc%26search...,1
42613,Fatal%3D0%26Severity%3D3%26Category%3D3%26Even...,1
43567,ara%3D%22%3E%3Cscript%3Ealert%28%22BackDoor%22...,1


# Preprocessing

In [123]:
"""
Finding the number of xss examples
xssed = df.apply(lambda x: True if 1 in list(x) else False, axis=1)
xss_count = len(xssed[xssed == True].index)
"""

In [ ]:
X=df['Code']
Y=df['Label']

In [ ]:
X.head()

73039    clef=crochesdor&titre=%22%3E%3Cscript%3Ealert%...
4960                  aid=/20011207/reviews/112070301/1023
96239    prod=%22%3E%27%3E%3C%73%63%72%69%70%74%3E%61%6...
60741    lg%3D%3C/script%3E%3Ciframe%20src%3Dhttp%3A//x...
71075    country%3D%22%3E%3Cscript%3Ealert%28document.c...
Name: Code, dtype: object

In [127]:
sentences=df['Code'].values
y=df['Label'].values

In [ ]:
# Function to convert the sentences(code) to ASCII values

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        
        """Some characters have very large values,
        thus removing letters having values greater than 8222 and for characters whose values lie between 
        128 and 8222 assigning them values so they can be normalized more easily."""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)

            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
            #If values less than 128 store them else discard them
          
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
    """       
    All data (except 1 string - the extra characters in this are truncated eventually)
    are less than 10000 characters, thus the zer array is initialised to the 10000 zeroes. 
    zer=np.zeros((10000))
    The minimum of length of zer and sentence_ascii is taken to fill the values of zer. 
    """
    
    for i in range(min(len(sentence_ascii), len(zer))):
        zer[i]=sentence_ascii[i]

    return zer.reshape(100, 100)


In [ ]:
# Sending each sentence to be converted to ASCII and the output is an image :

def make_image(sentences):
  arr=np.zeros((len(sentences),100,100))
  for i in range(len(sentences)):
      image=convert_to_ascii(sentences[i])
      x=np.asarray(image,dtype='float')
      image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
      image/=128
      arr[i]=image
  #Reshaping data for input to the CNN
  return arr.reshape(arr.shape[0], 100, 100, 1)

In [ ]:
#Average length of a data input 
#sum([len(i) for i in sentences]) / len(sentences)

88.90807812648147

In [ ]:
# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(sentences,y, test_size=0.2, random_state=42)

Using a data generator: Generating our dataset on multiple cores in real time and then feeding it to our deep learning model to deal with the memory-consuming data in a better way. 


In [ ]:
class DatasetObject(Sequence):

  def __init__(self, x, y, batch_size=64):
    self.x, self.y = x, y
    self.batch_size = batch_size

  #Returns the number of batches in the Sequence
  def __len__(self):
    return math.ceil(len(self.x) / self.batch_size)

  #Returning the image corresponding to the idx and the labels from the training set
  #Gets a complete batch at position idx
  def __getitem__(self, idx):                 
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
    return make_image(batch_x), batch_y

In [ ]:
train_ds = DatasetObject(trainX, trainY)
test_ds = DatasetObject(testX, testY)
len(train_ds), len(test_ds)

(1319, 330)

In [ ]:
train_ds[3][0].shape

(64, 100, 100, 1)

# Model

In [ ]:
# Total number of layers = 11
# Number of Convolutional layer: 3

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])   


In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 256)       295168    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25600)            

In [ ]:
# Stop when validation accuracy > 97 (Callbacks)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [ ]:
#On training, val_accuracy drops after the 7th epoch. 
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(train_ds,
                epochs=num_epoch,
                verbose=1,
                validation_data=test_ds,
 #               callbacks=[callbacks]
            )

Epoch 1/10
1319/1319 [==============================] - 56s 42ms/step - loss: 0.0472 - accuracy: 0.9863 - val_loss: 0.0401 - val_accuracy: 0.9885
Epoch 2/10
1319/1319 [==============================] - 56s 42ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0315 - val_accuracy: 0.9910
Epoch 3/10
1319/1319 [==============================] - 55s 42ms/step - loss: 0.0293 - accuracy: 0.9919 - val_loss: 0.0373 - val_accuracy: 0.9894
Epoch 4/10
1319/1319 [==============================] - 55s 42ms/step - loss: 0.0278 - accuracy: 0.9923 - val_loss: 0.0280 - val_accuracy: 0.9926
Epoch 5/10
1319/1319 [==============================] - 55s 42ms/step - loss: 0.0240 - accuracy: 0.9933 - val_loss: 0.0312 - val_accuracy: 0.9924
Epoch 6/10
1319/1319 [==============================] - 55s 42ms/step - loss: 0.0228 - accuracy: 0.9936 - val_loss: 0.0394 - val_accuracy: 0.9889
Epoch 7/10
1319/1319 [==============================] - 55s 42ms/step - loss: 0.0214 - accuracy: 0.9942 - val_loss: 0.0238 -

# Results

In [ ]:
# Prediction for test set:
from tqdm.notebook import tqdm
preds = [model.predict(test_ds[i][0]) for i in tqdm(range(len(test_ds)))]
preds = tf.concat(preds, axis=0).numpy()
preds.shape

(21094, 1)

In [ ]:
# Threshold values predicted:

for i in range(len(preds)):
    if preds[i]>0.5:
        preds[i]=1
    elif preds[i]<=0.5:
        preds[i]=0


In [ ]:
# Counting the true predicted and wrong predicted labels:

true=0
false=0

for i in range(len(preds)):
    if preds[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  20952
false prediction ::  142


In [ ]:
# Number of attack(XSS) and benign(normal) data in test set:

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  14740
 Benign data in test set ::  6354


In [ ]:
# Takes true positive, true negative, false positive and false negative as input:
# Returns Accuracy

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [ ]:
# Takes true positive and false positive as input:
# Returns Precision

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [ ]:
#Takes true positive and false negative as input:
# Returns Recall

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [ ]:
#Takes true values and the predicted values as input:
# Returns accuracy, precision and recall.

def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1
        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(testY,pred)


In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))


 Accuracy : 0.984375 
 Precision : 1.0 
 Recall : 0.9803921568627451
